**TCN Model**

In [1]:
import torch
import torch.nn as nn

class TCNModel(nn.Module):
    def __init__(self, input_channels, num_channels, num_classes):
        super(TCNModel, self).__init__()
        layers = []
        for i in range(len(num_channels)):
            conv_layer = nn.Conv2d(input_channels if i == 0 else num_channels[i-1],
                                   num_channels[i],
                                   kernel_size=(3, 3),
                                   padding=(1, 1))
            relu_layer = nn.ReLU()
            bn_layer = nn.BatchNorm2d(num_channels[i])
            pool_layer = nn.MaxPool2d(kernel_size=(2, 2))

            # Append named layers to the list
            layers.append((f'conv{i+1}', conv_layer))
            layers.append((f'relu{i+1}', relu_layer))
            layers.append((f'batch_norm{i+1}', bn_layer))
            layers.append((f'max_pool{i+1}', pool_layer))

        # Use OrderedDict to preserve layer names in nn.Sequential
        from collections import OrderedDict
        self.network = nn.Sequential(OrderedDict(layers))

        # Calculating the flattened output size dynamically
        dummy_input = torch.randn(1, input_channels, 128, 128)
        output_shape = self.network(dummy_input).view(1, -1).shape[1]

        self.fc = nn.Linear(output_shape, num_classes)  # Using the calculated output size

    def forward(self, x):
        x = self.network(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

num_channels = [32, 64, 128]

model = TCNModel(input_channels=1, num_channels, num_classes=2)
print(model)


TCNModel(
  (network): Sequential(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU()
    (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU()
    (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU()
    (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=32768, out_features=2, bias=True)